# Introduction
This Notebook will explore how to prompt Gemma 3, using KerasNLP.

## What is Gemma?
Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models. At the 3rd generation now, Gemma 3 comes in 4 sizes, 1B, 4B, 12B and 27B, both pretrained and instruction finetuned versions.   

Models 4B, 12B, 27B brings an extended context window (up to 128K) as well as multi-modality. 

The 1B model, although incredibly compact, is not only very fast but is also quite powerful.

We will demonstrate here the features of 1B model while running on CPU only.


# Prerequisites

## Install packages

We will install Keras and KerasNLP.

In [1]:
!pip install -q -U keras-nlp
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.3/731.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


## Import packages

In [2]:
import keras
import keras_nlp
from keras_nlp.samplers import TopKSampler
from time import time

Select the backend. Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Keras 3 lets you choose the backend: TensorFlow, JAX, or PyTorch. For this Notebook, we will choose jax as backend.

In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

## Initialize the model

Make sure to use `Gemma3CausalLM` to initialize the model, not the `GemmaCausalLM`. Otherwise the Gemma3 backbone will not be recognized.

In [4]:
gemma_lm = keras_nlp.models.Gemma3CausalLM.from_preset("/kaggle/input/gemma3/keras/gemma3_1b/1")

In [5]:
tokenizer = keras_nlp.models.Gemma3Tokenizer.from_preset("/kaggle/input/gemma3/keras/gemma3_1b/1")

Let's verify the model now.

In [6]:
gemma_lm.summary()

Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │     999,885,952 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 999,885,952 (3.72 GB)

 Trainable params: 999,885,952 (3.72 GB)

 Non-trainable params: 0 (0.00 B)

# Test the model

Let's define a simple prompt.

In [7]:
instructions = "You are an AI that answers in short, complete sentences only.\n"
prompt = (
    f"{instructions}"
    "Question: {question}\n"
    "Answer:"
)

output = gemma_lm.generate(
    prompt.format(question="What is the temperature of the Moon?"),
    max_length=40, 
    stop_token_ids=[tokenizer.token_to_id("\n")]
)


In [8]:
answer = output.replace(instructions, "").strip()
print(answer)

Question: What is the temperature of the Moon?
Answer: The temperature of the Moon is 100 degrees Celsius


## Functions to generate and format the output

We define a function to generate the answer.

In [9]:
def generate_answer(question, 
                    instructions="You are an AI that answers in short, complete sentences only.\n", 
                    max_length=40):
    prompt = (
        f"{instructions}"
        "Question: {question}\n"
        "Answer:"
    )
    output = gemma_lm.generate(
        prompt.format(question=question),
        max_length=max_length, 
        stop_token_ids=[tokenizer.token_to_id("\n")]
    )    
    answer = output.replace(instructions, "").strip()
    return answer

We define a function to format the output.

In [10]:
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Explanation", "Total time"], ["blue", "red", "green", "darkblue",  "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

Now let's combine both.

In [11]:
def generate_format_answer(question, 
                    instructions="You are an AI that answers in short, complete sentences only.\n", 
                    max_length=40):
    t = time()
    answer = generate_answer(question, instructions, max_length)
    display(Markdown(colorize_text(f"{answer}\n\nTotal time: {round(time()-t, 2)} sec.")))

In [12]:
generate_format_answer("What is the surface temperature of the Moon?")



**<font color='red'>Question:</font>** What is the surface temperature of the Moon?


**<font color='green'>Answer:</font>** 100 degrees Celsius



**<font color='magenta'>Total time:</font>** 2.63 sec.

## Let's ask some simple common knowledge questions

In [13]:
answer = generate_format_answer("When was the 30 years war?")



**<font color='red'>Question:</font>** When was the 30 years war?


**<font color='green'>Answer:</font>** 1618-1648



**<font color='magenta'>Total time:</font>** 3.63 sec.

In [14]:
answer = generate_format_answer("When was the Great War?")



**<font color='red'>Question:</font>** When was the Great War?


**<font color='green'>Answer:</font>** 1914-1918



**<font color='magenta'>Total time:</font>** 4.07 sec.

In [15]:
generate_format_answer("When was the USA-Spanish war?")



**<font color='red'>Question:</font>** When was the USA-Spanish war?


**<font color='green'>Answer:</font>** 1898



**<font color='magenta'>Total time:</font>** 2.36 sec.

In [16]:
generate_format_answer("Who was the president before 1st mandate of Donald Trump?")



**<font color='red'>Question:</font>** Who was the president before 1st mandate of Donald Trump?


**<font color='green'>Answer:</font>** Barack Obama



**<font color='magenta'>Total time:</font>** 1.57 sec.

In [17]:
generate_format_answer("When was the attack on Pearl Harbor?")



**<font color='red'>Question:</font>** When was the attack on Pearl Harbor?


**<font color='green'>Answer:</font>** December 7, 1941



**<font color='magenta'>Total time:</font>** 3.39 sec.

In [18]:
generate_format_answer("Who was the next shogon after Yeiatsu Tokugawa?")



**<font color='red'>Question:</font>** Who was the next shogon after Yeiatsu Tokugawa?


**<font color='green'>Answer:</font>** Tokugawa Ieyasu



**<font color='magenta'>Total time:</font>** 2.11 sec.

This answer is not correct. The correct answer is Hidetada Tokugawa.

In [19]:
generate_format_answer("Who was the first American president?")



**<font color='red'>Question:</font>** Who was the first American president?


**<font color='green'>Answer:</font>** George Washington



**<font color='magenta'>Total time:</font>** 1.54 sec.

## Let's explore more about technology



In [20]:
generate_format_answer("What is XGBoost?", 
                       max_length=50)



**<font color='red'>Question:</font>** What is XGBoost?


**<font color='green'>Answer:</font>** XGBoost is a machine learning algorithm that is used to solve classification problems. It is a gradient boosting algorithm that is used to



**<font color='magenta'>Total time:</font>** 25.93 sec.

In [21]:
generate_format_answer("What is AlphaFold?")



**<font color='red'>Question:</font>** What is AlphaFold?


**<font color='green'>Answer:</font>** AlphaFold is a computer program that predicts the structure of proteins. It is



**<font color='magenta'>Total time:</font>** 4.72 sec.

In [22]:
generate_format_answer("What is RandomForest?")



**<font color='red'>Question:</font>** What is RandomForest?


**<font color='green'>Answer:</font>** A machine learning algorithm that uses a combination of decision trees to make predictions.



**<font color='magenta'>Total time:</font>** 4.97 sec.

## Let's do math

In [23]:
generate_format_answer("What is 123 + 11?")



**<font color='red'>Question:</font>** What is 123 + 11?


**<font color='green'>Answer:</font>** 134

You are an AI



**<font color='magenta'>Total time:</font>** 3.14 sec.

In [24]:
generate_format_answer("What is 25 x 25?")



**<font color='red'>Question:</font>** What is 25 x 25?


**<font color='green'>Answer:</font>** 625



**<font color='magenta'>Total time:</font>** 2.07 sec.

# More math and Python code

In [25]:
prompt = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""

In [26]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to calculate the area of a circle of radius r"), max_length=128)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r


**<font color='green'>Answer:</font>**
def area(r):
return 3.14 * r * r
print(area(10))




**<font color='magenta'>Total time:</font>** 26.46 sec.

In [27]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to order a list"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to order a list


**<font color='green'>Answer:</font>**
def order_list(list):
for i in range(len(list)):
for j in range(i+1, len(list)):
if list[i] > list[j]:
list[i], list[j] = list[j], list[i]
return list




**<font color='magenta'>Total time:</font>** 40.62 sec.

# Conclusions


The Gemma 3 model (1B) from Google DeepMind is powerful for its size and can answer to a lot of questions about history, politics, as well as simple math. It is not great with writing code and formating the output when asked to write code.